## VeloBall Demonstration

I'll use this notebook to demonstrate the application of VeloBall to help inform professional cycling teams about the expected payoffs associated with sending a rider with a certain profile to a specific race.

### A brief note on context
Cycling isn't popular compared the top sports, but in Europe, professional cycling teams across multiple divisions are generally funded by a myriad of sponsors who seek to gain exposure and attention through success from the team they sponsor. There are a number of ways that a team/rider can "succeed" in cycling races. A rider can place well on a given day. One-off cycling results are extremely difficult to predict, as the specifics of a given course and in-race tactics (in addition to luck) can significantly influence race results. The other measure of "success" is the general classification (GC). GC is specific to "stage races," which are effectively races contested over multiple individual races (stages), each of which with its own winner. The rider with the smallest aggregate time across all the stages of the race is the winner of that race's GC. The Tour de France is a stage race with 21 stages. When people discuss the winner of the Tour de France, they are referring to the winner of the GC of the Tour de France.

As I mentioned above, sponsors of cycling teams are seeking exposure to the general public through the results of their teams. However, these teams have limited rosters and certain riders are only capable of certain things. Spefically, many teams find themselves sending riders to races seeking top GC results when this may not be realistic. Sending a rider to a race such as the Tour de France brings with it a significant opportunity cost, as contesting the GC of a race such as the Tour de France is a 21-day commitment for the rider, and significantly hampers their ability to attempt to win individual stages of the race, or to have success at other races which precede or postcede the Tour. 

There are a number of ways one can measure the overall performance of a cycling team over the course of a system. One of these is UCI Points, the official scoring system of cycling's international governing body. Professional road cycling recently introduced a promotion/relegation system (like that of pro soccer), with important ramifications for teams with regards to race invites and sponsor dollars. UCI Points govern the promotion and relegation of teams, and thus are extremely important. <b> Thus, this first iteration of VeloBall will focus on providing a team with an expected value of points scored from GC by a rider from one of cycling's three 3-week grand tours, which offer the largest amounts of UCI Points for GC, but also are the most hotly contested races. This will help a team determine if it is "worth it" for one of their riders to contest GC at that grand tour, or if they should focus on other things. Such a determination is something teams often struggle with. </b>

In [72]:
# ===== Imports ===== #
import ast
import pandas as pd

## UCI Points
As mentioned above, UCI Points will be my performance metric of choice for my demonstration of VeloBall. It is not the only performance metric one could use, but it is certainly one which teams will care about, as UCI Points are directly tied to the team's status and the races it gets invited to, and thus affect sponsorship dollars.

As the VeloBall model is for predicting probabilitiesfor finishing in the top 10 of grand tours (or finishing outside the top 10), we will only concern ourselves with the points available for finishing in the top 10 of cycling's three grand tours: the Tour de France (biggest race in cycling), Giro d'Italia (second biggest race in cycling), and the Vuelta a Espana (third biggest race in cycling). The Tour de France has its own points scale and is worth the most across each finishing position, and the Giro d'Italia and Vuelta a Espana are subsequently worth the same amount at each corresponding position.

In [90]:
# ===== Tour de France UCI points through 10 positions ===== #
# NOTE: 0 points are given for this demonstration if a rider finishes outside the top 10
# (as this is what the model allows), but in reality there also are points
# available for the next few positions outsid the top 10, but they are less important.
tdf_points = {
    0: 1000,  # points allotted for Tour de France victory
    1: 800, # points for second place
    2: 675,
    3: 575,
    4: 475,
    5: 400,
    6: 325,
    7: 275,
    8: 225,
    9: 175,
    10: 0  # 0 points allotted for positions outside the top 10, for demonstration purposes
}

# ===== Giro and Vuelta UCI Points through 10 positions
gv_points = {
    0: 850,
    1: 680,
    2: 575,
    3: 460,
    4: 380,
    5: 320,
    6: 260,
    7: 220,
    8: 180,
    9: 140,
    10: 0
}

### Training Data
I give a more detailed description of the training data in my report, but I load the training data here as a reference for the user. The training data include rider characteristics such as their previous performances in grand tours, and two different Elo rating metrics (briefly described below), which encode information about the rider's general GC riding ability. Further, the training data also included relative features about a rider's competition, such as the difference between a rider's general Elo rating and the general GC Elo rating of the top-rated rider entered in the race. <b> These relative features (1_diff, 3_diff, 5_diff, 10_diff) were some of the most important features for VeloBall's Watson AutoAI XGBoost model, with a rider's general GC Elo rating also being one of the most important. </b>

#### Elo Ratings
The generalized GC Elo rating ("elo_rating" column) and the TT Elo rating ("tt_elo_rating" column) are metrics I developed myself outside of this class. I used them for my VeloBall project as I thought they'd make informative features for a model such as the one I'm presenting here. Elo Ratings were originally devised as a rating system for chess (see here: https://en.wikipedia.org/wiki/Elo_rating_system#:~:text=The%20Elo%20rating%20system%20is,a%20Hungarian%2DAmerican%20physics%20professor), but I altered the algorithm so that it worked for a multiplayer game such as cycling.

In [69]:
# ===== Training data ===== #
# Here, each sample corresponds to a rider who entered one of cycling's 3 grand tours.
# The target column is the "actual" feature, which encodes the result of the rider in
# that grand tour. My VeloBall model is a multiclass classifier, with class 0 encoding
# victory, class 1 encoding second place, etc. Class 9 therefore encodes 10th place, and
# class 10 encompasses any result outside of the top 10.
train_df = pd.read_csv('veloball-demonstration-training.csv')
print(f'Number of training examples: {len(train_df.index)}')
train_df.head(10)

Number of training examples: 8960


,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s,actual
0,18,1562.952640,29,1519.260125,135.815804,109.047382,96.591855,40.455111,192.362656,124.855071,99.656768,36.706587,89.3,0,0,0,10
1,47,1499.848839,80,1494.635758,385.851873,136.632172,114.739055,84.952918,140.212232,108.097086,104.088800,73.520876,60.5,0,0,0,10
2,14,1558.410014,101,1500.000000,156.713437,140.880633,116.630695,39.947812,315.731768,156.795108,89.241343,47.020819,46.0,0,0,0,10
3,70,1502.364515,99,1499.559759,286.366431,225.713534,193.485194,118.291762,208.448729,184.045917,99.605830,76.977192,101.4,0,0,0,10
4,93,1490.489867,101,1500.000000,161.998933,136.370032,91.929046,67.731755,174.664936,114.507962,66.161256,46.792627,64.9,0,0,0,10
5,75,1495.161719,24,1502.389840,145.445618,81.090014,48.505065,32.800092,114.222469,32.951416,27.697494,15.875456,58.9,0,0,0,10
6,60,1506.532718,67,1510.236141,312.726711,125.333951,116.860793,92.231125,147.362348,108.664087,103.948122,63.636546,74.0,0,0,0,10
7,41,1508.116403,101,1500.000000,182.827066,120.659260,100.741388,64.407955,145.403818,60.381976,53.313543,43.969880,34.9,0,0,0,10
8,97,1494.366985,97,1497.081646,287.260481,162.907084,139.099548,87.089702,252.560238,130.442890,112.450602,60.014417,47.0,0,0,0,10
9,16,1535.326689,37,1504.006266,122.184530,65.292932,54.627596,18.154489,171.603525,80.953325,66.215360,47.265791,46.3,0,0,0,7


## Using the VeloBall Model
I unfortunately ran out of free Watson ML compute resources during this project, and thus can't demonstrate a running of the VeloBall model using Watson's Python library. Instead, I will load the result of a dataset of 150 rider profiles I fed to the model as a batch and demonstrate how VeloBall would be used, using that model output.

Note that due to the multiclass nature of VeloBall's prediction problem, the output probability of a rider finishing in a given position will never be particularly high. Further, due to a number of factors including team dynamics, crashes, sickness, etc., highly rated riders who enter a grand tour as strong GC contenders may still fail to finish in the top 10. Thus, even for the best riders, their given probability of finishing outside the top 10 will be higher than expected. This is a feature, not a bug. However, <b> this also leads the prediction label output by the model to be off, as the probability of finishing outside the top 10 will almost always be the class with the highest given probability. For the purposes of VeloBall, we will only concern ourselves with the class probabilities output by the VeloBall model. </b>

In [79]:
# ===== Load the example VeloBall output ===== #
out = pd.read_csv('veloball-output-example.csv')

# ==== Drop the prediction column and convert probability column to lists ===== #
out = out.drop(columns = 'prediction')
out['probability'] = [ast.literal_eval(probs) for probs in out['probability']]
out.head(5)

,rider,probability
0,BERNAL Egan,"[0.12177615612745285, 0.07134881615638733, 0.1..."
1,YATES Simon,"[0.07867355644702911, 0.06937272101640701, 0.0..."
2,CARTHY Hugh,"[0.08079802989959717, 0.06374413520097733, 0.0..."
3,VLASOV Aleksandr,"[0.08355908840894699, 0.06701226532459259, 0.0..."
4,MARTIN Dan,"[0.07068194448947906, 0.06586537510156631, 0.0..."


## Demonstration Data
I will now load the sample data of 150 examples I fed to the VeloBall model to obtain the output contained in the "out" variable (defined in the previous cell).

Each sample in the demonstration data corresponds to the same row in the "out" dataframe. The 150 samples in the demonstration data contain the profiles of the 50 top GC riders (by Elo rating) who entered each of the three grand tours in 2021. The samples are ordered, so the first 50 samples are the top 50 riders (in order) who entered the 2021 Giro d'Italia (Tour of Italy), second 50 are the top 50 entrants into the 2021 Tour de France, and the final 50 are the top 50 entrants to the 2021 Vuelta a Espana (Tour of Spain). 

In [85]:
# ===== Demonstration data ===== #
demonstration = pd.read_csv('veloball-demonstration-test.csv')

### Profiles for the top 10 entrants to the 2021 Giro d'Italia

In [86]:
demonstration.iloc[0: 10, :]

,rider,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s
0,BERNAL Egan,1,1682.304186,35,1512.247219,0.000000,-58.458170,-80.398159,-103.271045,146.608379,62.029901,41.678687,20.611722,38.9,1,0,0
1,YATES Simon,2,1631.703674,12,1530.367869,50.600512,-7.857658,-29.797647,-52.670533,128.487730,43.909252,23.558037,2.491073,38.9,1,0,3
2,CARTHY Hugh,3,1623.846017,22,1522.899170,58.458170,0.000000,-21.939989,-44.812875,135.956429,51.377951,31.026736,9.959772,38.9,0,1,0
3,VLASOV Aleksandr,4,1606.352911,49,1505.594827,75.951275,17.493105,-4.446884,-27.319770,153.260771,68.682293,48.331078,27.264114,38.9,0,0,0
4,MARTIN Dan,5,1601.906027,36,1512.017157,80.398159,21.939989,0.000000,-22.872886,146.838441,62.259963,41.908748,20.841784,38.9,0,0,5
5,ALMEIDA João,6,1594.245359,6,1546.881950,88.058827,29.600658,7.660668,-15.212218,111.973648,27.395170,7.043956,-14.023009,38.9,0,0,1
6,NIBALI Vincenzo,7,1591.270247,15,1527.359910,91.033940,32.575770,10.635781,-12.237105,131.495689,46.917211,26.565996,5.499032,38.9,4,7,4
7,BILBAO Pello,8,1588.065913,17,1527.261628,94.238274,35.780104,13.840115,-9.032771,131.593971,47.015493,26.664278,5.597314,38.9,0,0,2
8,CARUSO Damiano,9,1580.204347,13,1529.053547,102.099840,43.641670,21.701680,-1.171206,129.802052,45.223574,24.872359,3.805395,38.9,0,0,3
9,BARDET Romain,10,1579.033141,50,1503.729627,103.271045,44.812875,22.872886,0.000000,155.125972,70.547494,50.196279,29.129315,38.9,0,2,3


### Profiles for the top 10 entrants to the 2021 Tour de France

In [88]:
demonstration.iloc[50: 60, :]

,rider,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s
50,POGAČAR Tadej,1,1820.859187,6,1578.590555,0.000000,-96.116517,-181.747910,-211.471365,60.932244,16.782284,1.645870,-16.522202,58.0,1,1,0
51,PORTE Richie,2,1755.062157,8,1570.100791,65.797030,-30.319487,-115.950880,-145.674335,69.422009,25.272048,10.135634,-8.032438,58.0,0,1,2
52,CARAPAZ Richard,3,1724.742670,26,1527.440475,96.116517,0.000000,-85.631392,-115.354848,112.082324,67.932363,52.795949,34.627877,58.0,1,1,1
53,THOMAS Geraint,4,1708.883641,3,1595.372839,111.975546,15.859029,-69.772364,-99.495819,44.149960,0.000000,-15.136414,-33.304487,58.0,1,1,0
54,URÁN Rigoberto,5,1639.111278,14,1551.552629,181.747910,85.631392,0.000000,-29.723455,87.970170,43.820210,28.683796,10.515724,58.0,0,3,5
55,KELDERMAN Wilco,6,1633.140821,15,1542.098559,187.718367,91.601849,5.970457,-23.752998,97.424240,53.274280,38.137865,19.969793,58.0,0,1,4
56,VALVERDE Alejandro,7,1625.564889,37,1519.589267,195.294298,99.177781,13.546388,-16.177067,119.933533,75.783572,60.647158,42.479086,58.0,1,8,11
57,MAS Enric,8,1625.497119,31,1523.169256,195.362068,99.245551,13.614159,-16.109297,116.353543,72.203582,57.067168,38.899096,58.0,0,1,2
58,MARTIN Dan,9,1620.295908,63,1505.859272,200.563279,104.446762,18.815370,-10.908085,133.663527,89.513567,74.377152,56.209080,58.0,0,0,6
59,IZAGIRRE Ion,10,1609.387822,16,1542.078174,211.471365,115.354848,29.723455,0.000000,97.444625,53.294665,38.158250,19.990178,58.0,0,0,1


### Profiles for the top 10 entrants to the 2021 Vuelta a Espana

In [89]:
demonstration.iloc[100: 110, :]

,rider,elo_ranking,elo_rating,tt_elo_ranking,tt_elo_rating,1_diff,3_diff,5_diff,10_diff,tt_1_diff,tt_3_diff,tt_5_diff,tt_10_diff,tt_kms,grand_tour_wins,grand_tour_podiums,grand_tour_top10s
100,BERNAL Egan,1,1850.197456,25,1516.323828,0.000000,-119.036389,-167.205926,-258.138136,178.129956,33.347574,24.863718,18.252690,40.9,2,0,0
101,ROGLIČ Primož,2,1781.597897,1,1694.453784,68.599560,-50.436829,-98.606366,-189.538576,0.000000,-144.782382,-153.266239,-159.877266,40.9,2,2,1
102,CARUSO Damiano,3,1731.161068,7,1536.116139,119.036389,0.000000,-48.169537,-139.101747,158.337645,13.555263,5.071407,-1.539621,40.9,0,1,3
103,YATES Adam,4,1691.756925,14,1527.502759,158.440531,39.404142,-8.765395,-99.697605,166.951025,22.168643,13.684787,7.073759,40.9,0,0,3
104,MAS Enric,5,1682.991530,11,1529.172488,167.205926,48.169537,0.000000,-90.932210,165.281297,20.498914,12.015058,5.404031,40.9,0,1,3
105,MARTIN Guillaume,6,1646.599670,101,1500.000000,203.597787,84.561398,36.391861,-54.540349,194.453784,49.671402,41.187545,34.576518,40.9,0,0,1
106,BARDET Romain,7,1628.511276,44,1503.675662,221.686180,102.649792,54.480254,-36.451956,190.778123,45.995740,37.511884,30.900856,40.9,0,2,4
107,IZAGIRRE Ion,8,1603.430121,8,1534.730623,246.767335,127.730947,79.561410,-11.370801,159.723161,14.940779,6.456922,-0.154105,40.9,0,0,1
108,KRUIJSWIJK Steven,9,1597.430732,5,1541.187545,252.766724,133.730336,85.560798,-5.371412,153.266239,8.483856,0.000000,-6.611027,40.9,0,1,6
109,POELS Wout,10,1592.059320,29,1512.641821,258.138136,139.101747,90.932210,0.000000,181.811964,37.029581,28.545725,21.934698,40.9,0,0,2


## VeloBall Case Studies
I will now take the user through a number of case studies of riders who rode grand tours in 2021, showing how the VeloBall model can be used to calculate the expected number of points scored by the rider in that grand tour. Expected value is a simple formula which outputs a single "expected" reward, based on the probabilities of receiving different rewards. I will also provide context as to that rider's ultimate result in the grand tour and the actual number of points they scored.

<b> NOTE: The primary output (product) of VeloBall is the expected number of points scored by a rider in a race, which can be used by teams to better plan which races their riders target. This notebook specifically demonstrates this functionality with respect to the GC finishes of riders in grand tours, as that is all I had time for in this specific project. There is a lot of opportunity here for expansion </b>

One last note. It is important to recognize that the model takes into account the fact that even the best riders often fail to finish in the top 10 for factors such as crashes, sickness, bad luck, etc., and thus the probability of finishing outside the top 10 for the best riders is higher than one might expect. Since the model takes this into account, the difference between expected points and actual points scored will be relatively high if the rider goes on to win the race. <b> This is a feature not a bug, as it is important for teams to realize this relatively high failure rate, even for the best of the best. </b>

### Case Study 1: Egan Bernal at the 2021 Giro d'Italia
Egan Bernal is a Columbian cyclist who rides for the Ineos Grenadiers team. In 2019, he won the Tour de France, the biggest race in professional cyclist, and in the process became the first South American winner of the Tour and youngest winner in the modern era. During his defence of the Tour in 2020, he was forced to abandon the race 2 weeks in due to lingering back problems that had been plaguing him. The 2021 Giro d'Italia, which Bernal ultimately won, marked his comeback and triumph over the injuries that plagued his 2020 season.

Bernal scored 850 UCI points for winning the 2021 Giro d'Italia.

In [120]:
rider_index = 0

# ===== Print Bernal's feature profile leading into the race ===== #
print("Egan Bernal's profile:")
for feature in demonstration.columns:
    if feature != 'rider':
        print(f'\t{feature}: {round(float(demonstration.loc[rider_index, feature]), 1)}')
        
# ===== Print the probability of each finishing place for Bernal, as given by the VeloBall model ===== #
print("\n\nBernal's Giro finish probabilities, as given by VeloBall's model:")
probs = out.iloc[rider_index, 1]
for i in range(len(probs)):
    if i < 10:
        print(f'\t{i + 1}. {round(probs[i] * 100, 2)}%')
    else:
        print(f'\tOutside top 10: {round(probs[i] * 100, 2)}%')

# ===== Bernal's expected points at the 2021 Giro ===== #
expected_points = 0
for i in range(len(probs)):
    
    # gv_points are the points for each finishing position at the Giro and Vuelta; 
    # this dict is defined at the top of the notebook
    expected_points += (probs[i] * gv_points[i])

print(f"\n\nBernal's expected points: {round(expected_points, 2)}")

Egan Bernal's profile:
	elo_ranking: 1.0
	elo_rating: 1682.3
	tt_elo_ranking: 35.0
	tt_elo_rating: 1512.2
	1_diff: 0.0
	3_diff: -58.5
	5_diff: -80.4
	10_diff: -103.3
	tt_1_diff: 146.6
	tt_3_diff: 62.0
	tt_5_diff: 41.7
	tt_10_diff: 20.6
	tt_kms: 38.9
	grand_tour_wins: 1.0
	grand_tour_podiums: 0.0
	grand_tour_top10s: 0.0


Bernal's Giro finish probabilities, as given by VeloBall's model:
	1. 12.18%
	2. 7.13%
	3. 10.76%
	4. 7.27%
	5. 6.83%
	6. 5.97%
	7. 6.25%
	8. 5.0%
	9. 4.41%
	10. 4.43%
	Outside top 10: 29.76%


Bernal's expected points: 333.82


## Case Study 2: Enric Mas at the 2021 Tour de France
Enric Mas is a consistent if unspectacular GC rider, riding for the Movistar team, which has experienced performance troubles over the past couple of years, and thus was in great need to maximize their point totals in 2021. This is the type of team which VeloBall would be most useful for. Enric Mas is one of Movistar's best riders, and thus would need him to score maximum points. VeloBall's model gave an expected points of 185.73, if Mas chose to contest GC at the 2021 Tour de France, a number of points which would give the team a significant boost. Thus, if Movistar were to use VeloBall, they would be able to make a more informed decision about whether it was worth it for Mas to contest GC at the Tour, which in this case, it was. Ultimately, Mas finished 6th and scored 400 points for Movistar.

In [119]:
rider_index = 57

# ===== Print Mas' feature profile leading into the race ===== #
print("Enric Mas' profile:")
for feature in demonstration.columns:
    if feature != 'rider':
        print(f'\t{feature}: {round(float(demonstration.loc[rider_index, feature]), 1)}')
        
# ===== Print the probability of each finishing place for Mas, as given by the VeloBall model ===== #
print("\n\nMas' Tour finish probabilities, as given by VeloBall's model:")
probs = out.iloc[rider_index, 1]
for i in range(len(probs)):
    if i < 10:
        print(f'\t{i + 1}. {round(probs[i] * 100, 2)}%')
    else:
        print(f'\tOutside top 10: {round(probs[i] * 100, 2)}%')

# ===== Mas' expected points at the 2021 Tour ===== #
expected_points = 0
for i in range(len(probs)):
    
    # gv_points are the points for each finishing position at the Tour; 
    # this dict is defined at the top of the notebook
    expected_points += (probs[i] * tdf_points[i])

print(f"\n\nMas' expected points: {round(expected_points, 2)}")

Enric Mas' profile:
	elo_ranking: 8.0
	elo_rating: 1625.5
	tt_elo_ranking: 31.0
	tt_elo_rating: 1523.2
	1_diff: 195.4
	3_diff: 99.2
	5_diff: 13.6
	10_diff: -16.1
	tt_1_diff: 116.4
	tt_3_diff: 72.2
	tt_5_diff: 57.1
	tt_10_diff: 38.9
	tt_kms: 58.0
	grand_tour_wins: 0.0
	grand_tour_podiums: 1.0
	grand_tour_top10s: 2.0


Mas' Tour finish probabilities, as given by VeloBall's model:
	1. 2.8%
	2. 6.15%
	3. 3.34%
	4. 4.74%
	5. 6.93%
	6. 5.35%
	7. 5.4%
	8. 4.21%
	9. 3.89%
	10. 3.81%
	Outside top 10: 53.38%


Mas' expected points: 225.87


## Case Study 3: Nick Schultz at the 2021 Vuelta a Espana
Nick Schultz is a rider for the BikeExchange team. BikeExchange, like Movistar, have been struggling recently and are at risk of relegation from the UCI WorldTour. In organizing its roster for the Vuelta a Espana, maximizing points would have been a critical goal for the team. Nick Schultz is a rider who, at his best, may be able to contest a top 10 position on GC at the Vuelta, but who also may better serve his team by focusing on individual stage wins, which also can be used to accrue points. Based on Schultz's rider profile and the field at the 2021 Vuelta, VeloBall gave Schultz an expected points of 64.84 if he chose to contest GC at the Vuelta. This is not enough to justify Schultz spending 21 stages of the Vuelta attempting to finish in the top 10, so if BikeExchange were using VeloBall, they'd have an easier decision optimizing their roster for the Vuelta by sending Nick Schultz to hunt individual stage results, rather than contest GC.

In [117]:
rider_idx = 126

# ===== Print Schultz's feature profile leading into the race ===== #
print("Nick Schultz's profile:")
for feature in demonstration.columns:
    if feature != 'rider':
        print(f'\t{feature}: {round(float(demonstration.loc[rider_idx, feature]), 1)}')
        
# ===== Print the probability of each finishing place for Schultz, as given by the VeloBall model ===== #
print("\n\nSchultz's Vuelta finish probabilities, as given by VeloBall's model:")
probs = out.iloc[rider_idx, 1]
for i in range(len(probs)):
    if i < 10:
        print(f'\t{i + 1}. {round(probs[i] * 100, 2)}%')
    else:
        print(f'\tOutside top 10: {round(probs[i] * 100, 2)}%')

# ===== Schultz's expected points at the 2021 Vuelta ===== #
expected_points = 0
for i in range(len(probs)):
    
    # gv_points are the points for each finishing position at the Giro and Vuelta; 
    # this dict is defined at the top of the notebook
    expected_points += (probs[i] * gv_points[i])

print(f"\n\nSchultz's expected points: {round(expected_points, 2)}")

Nick Schultz's profile:
	elo_ranking: 27.0
	elo_rating: 1524.4
	tt_elo_ranking: 101.0
	tt_elo_rating: 1500.0
	1_diff: 325.8
	3_diff: 206.8
	5_diff: 158.6
	10_diff: 67.7
	tt_1_diff: 194.5
	tt_3_diff: 49.7
	tt_5_diff: 41.2
	tt_10_diff: 34.6
	tt_kms: 40.9
	grand_tour_wins: 0.0
	grand_tour_podiums: 0.0
	grand_tour_top10s: 0.0


Schultz's Vuelta finish probabilities, as given by VeloBall's model:
	1. 1.47%
	2. 1.5%
	3. 1.49%
	4. 1.56%
	5. 1.54%
	6. 1.95%
	7. 1.48%
	8. 1.54%
	9. 2.3%
	10. 2.14%
	Outside top 10: 83.03%


Schultz's expected points: 64.84
